#### How to transform odds files

1. Download files 
1. Get file path to downloaded file
1. Run the imports cell
1. Run the pipeline functions cell
1. Update column details for sport `team_cols` and `gm_periods`
1. Pass file path, `team_cols` and `gm_periods` to the `odds_pipeline` function.
1. If you want to save the transformed file call `to_csv` on the output of `odds_pipeline` 
1. Pass the argument `index=False` unless you want to include the index.

## Imports

In [318]:
import pandas as pd
import numpy as np

## Pipeline Functions

In [321]:
def load_data(path):
    xlsx = pd.ExcelFile(path)
    return pd.read_excel(xlsx)


def gen_gid(df):
    assert df.ndim == 2, "only pass dataframes"
    n_games = int(df.shape[0] / 2)
    games = np.arange(1, n_games + 1)
    games_2x = np.hstack([games, games])
    games_2x.sort()

    return df.assign(gid=games_2x)


def fix_neutral_games(df):
    neutral_games = df.loc[(df['VH'] == 'N'), 'gid'].unique()
    is_even = (df['Rot'] % 2 == 0).values
    df.loc[~is_even, 'VH'] = 'V'
    df.loc[is_even, 'VH'] = 'H'
    v = df.query("VH == 'V'").reset_index(drop=True)
    h = df.query("VH == 'H'").reset_index(drop=True)
    n_games = h['gid'].isin(neutral_games).values * 1
    h = h.assign(neutral=n_games)
    return h, v


def clean_odds_vals(df):
    assert df.ndim == 2, "pass dataframe only"
    return df.replace({
        'Open': {
            'pk': 0,
            'PK': 0,
            'nl': np.nan
        },
        'Close': {
            'pk': 0,
            'PK': 0,
            'nl': np.nan
        },
        '2H': {
            'pk': 0,
            'PK': 0,
            'nl': np.nan
        }
    }).astype({
        'Open': float,
        'Close': float,
        '2H': float
    })


def get_fav(odd_type):
    return 'h' if odd_type == 'OU' else 'v'


def get_ou(fav, h, v):
    return v if fav == 'h' else h


def get_sprd(fav, h, v):
    return h if fav == 'h' else v


def flip_odds(val, cutoff=60):
    return 'OU' if val > cutoff else 'Sprd'


ou_vec = np.vectorize(get_ou)
sprd_vec = np.vectorize(get_sprd)


def make_team_odds(h, v):
    assert h.ndim == 2, 'Pass dataframes only'
    assert v.ndim == 2, 'Pass dataframes only'
    h_odds = (h.loc[:, ['Open', 'Close', 'ML', '2H']].rename(columns={
        'Open': 'h_open',
        'Close': 'h_close',
        'ML': 'h_ml',
        '2H': 'h_2h'
    }))

    v.loc[:, 'open_type'] = v['Open'].map(flip_odds)
    v.loc[:, 'close_type'] = v['Close'].map(flip_odds)
    v.loc[:, 'h2_type'] = v['2H'].map(flip_odds)
    v.loc[:, 'open_fav'] = v['open_type'].map(get_fav)
    v.loc[:, 'close_fav'] = v['close_type'].map(get_fav)
    v.loc[:, 'h2_fav'] = v['h2_type'].map(get_fav)
    v_cols = [
        'open_type', 'close_type', 'h2_type', 'open_fav', 'close_fav',
        'h2_fav', 'Open', 'Close', 'ML', '2H'
    ]
    v_odds = v.loc[:, v_cols].copy().rename(columns={
        'Open': 'v_open',
        'Close': 'v_close',
        'ML': 'v_ml',
        '2H': 'v_2h'
    })
    return pd.concat([h_odds, v_odds], axis=1)


def reformat_odds(odds):
    assert odds.ndim == 2, 'Pass dataframes only'

    odds['open_ou'] = ou_vec(odds['open_fav'], odds['h_open'], odds['v_open'])
    odds['close_ou'] = ou_vec(odds['close_fav'], odds['h_close'],
                              odds['v_close'])
    odds['h2_ou'] = ou_vec(odds['h2_fav'], odds['h_2h'], odds['v_2h'])
    odds['open_sprd'] = sprd_vec(odds['open_fav'], odds['h_open'],
                                 odds['v_open'])
    odds['close_sprd'] = sprd_vec(odds['close_fav'], odds['h_close'],
                                  odds['v_close'])
    odds['h2_sprd'] = sprd_vec(odds['h2_fav'], odds['h_2h'], odds['v_2h'])
    odds_cols = [
        'h_ml', 'v_ml', 'open_sprd', 'open_fav', 'close_sprd', 'close_fav',
        'open_ou', 'close_ou', 'h2_sprd', 'h2_fav', 'h2_ou'
    ]
    return odds.loc[:, odds_cols].copy()


def col_select_rename(df, cols, prefix):
    assert df.ndim == 2, 'Pass dataframes only'
    df = df.loc[:, cols]
    df_cols = df.columns.tolist()
    new_cols = [f'{prefix}{col.lower()}' for col in df_cols]

    df.columns = new_cols
    return df


def get_game_details(h, v, team_cols, gm_periods):
    h_info = col_select_rename(h, team_cols + gm_periods, 'h_')
    v_info = col_select_rename(v, team_cols + gm_periods, 'v_')
    g_info = col_select_rename(h, ['gid', 'neutral', 'Date'], '')
    g_info.loc[:, 'ot'] = is_ot(h, gm_periods) * 1
    return pd.concat([g_info, h_info, v_info], axis=1)


def is_ot(df, gm_periods):
    score = col_select_rename(df, gm_periods, '').sum(axis=1)
    return df['Final'] > score


def odds_pipeline(path, team_cols, gm_periods):
    df = load_data(path)
    assert df.ndim == 2, 'Not a dataframe'
    assert len(team_cols) > 1, 'Provide team columns from raw data'
    assert len(gm_cols) > 1, 'Provide game info columns from raw data'
    h, v = gen_gid(df).pipe(clean_odds_vals).pipe(fix_neutral_games)
    odds = make_team_odds(h, v).pipe(reformat_odds)
    assert odds.ndim == 2, 'Pass dataframes only'
    games = get_game_details(h, v, team_cols, gm_periods)
    return pd.concat([games, odds], axis=1)

## Provide Column Names

Get the column names from the raw data `df.columns`

- `team_cols` = rotation number, team name, and final score
- `gm_periods` the split format of the sport using the column names as reference. For example, NBA periods are `['1st','2nd','3rd','4th']`, whereas NCAAB is `['1st','2nd']`

In [300]:
team_cols = ['Rot','Team', 'Final']
gm_periods = ['1st','2nd','3rd','4th']

## Transform Data

In [ ]:
nba_path = 'path/to/my/file.csv'

In [304]:
game_odds = odds_pipeline(nba_path, team_cols, gm_periods)

In [319]:
game_odds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312 entries, 0 to 1311
Data columns (total 29 columns):
gid           1312 non-null int64
neutral       1312 non-null int64
date          1312 non-null int64
ot            1312 non-null int64
h_rot         1312 non-null int64
h_team        1312 non-null object
h_final       1312 non-null int64
h_1st         1312 non-null int64
h_2nd         1312 non-null int64
h_3rd         1312 non-null int64
h_4th         1312 non-null int64
v_rot         1312 non-null int64
v_team        1312 non-null object
v_final       1312 non-null int64
v_1st         1312 non-null int64
v_2nd         1312 non-null int64
v_3rd         1312 non-null int64
v_4th         1312 non-null int64
h_ml          1312 non-null int64
v_ml          1312 non-null int64
open_sprd     1312 non-null float64
open_fav      1312 non-null object
close_sprd    1312 non-null float64
close_fav     1312 non-null object
open_ou       1312 non-null float64
close_ou      1312 non-null float

In [320]:
game_odds.head()

,gid,neutral,date,ot,h_rot,h_team,h_final,h_1st,h_2nd,h_3rd,...,v_ml,open_sprd,open_fav,close_sprd,close_fav,open_ou,close_ou,h2_sprd,h2_fav,h2_ou
0,1,0,1016,0,502,Boston,105,21,26,30,...,170,5.0,h,4.5,h,208.5,211.5,1.0,h,106.0
1,2,0,1016,0,504,GoldenState,108,31,26,26,...,711,11.5,h,12.0,h,223.5,220.5,4.5,h,108.5
2,3,0,1017,0,702,Indiana,111,27,29,20,...,263,7.0,h,7.5,h,209.0,204.5,1.0,v,99.0
3,4,0,1017,0,704,Charlotte,112,23,31,29,...,-165,1.5,v,3.0,v,217.0,222.0,2.5,h,114.0
4,5,0,1017,0,706,Orlando,104,25,29,25,...,-135,2.0,v,2.5,v,210.5,208.0,3.0,v,103.0


## Export Data

In [ ]:
game_odds.to_csv('odds_trns.csv', index = False)